In [1]:
%run "ZeroMQ Python Connector".py
%load_ext autoreload
%autoreload
%matplotlib notebook
import pandas as pd

In [2]:
#Create connector instance, with particular port for PUSH, PULL and SUB
connector = DWX_ZeroMQ_Connector()

[INIT] Ready to send commands to METATRADER (PUSH): 32768
[INIT] Listening for responses from METATRADER (PULL): 32769
[INIT] Listening for market data from METATRADER (SUB): 32770


In [3]:
ID = connector._DWX_MTX_SEND_RATESDATA_REQUEST_(_symbol='GBPUSD', _start=(Timestamp.now()-Timedelta(days=100)).strftime('%Y.%m.%d %H:%M:00'),_timeframe=1440,_end=Timestamp.now().strftime('%Y.%m.%d %H:%M:00'))
response = connector._get_response_(messageID = ID)
print("\n This is the response: ", response)

{'_messageID': 302026415, '_action': 'DATA', '_data': {'2019.02.18 00:00': [1.29041, 1.29387, 1.28916, 1.29223, 114717, 0, 0], '2019.02.19 00:00': [1.29176, 1.3073, 1.28956, 1.30609, 120573, 0, 0], '2019.02.20 00:00': [1.30602, 1.3109, 1.30117, 1.30488, 126974, 0, 0], '2019.02.21 00:00': [1.30461, 1.30951, 1.3026, 1.30371, 135619, 0, 0], '2019.02.22 00:00': [1.30248, 1.308, 1.29679, 1.3054, 127336, 0, 0], '2019.02.25 00:00': [1.30499, 1.31144, 1.30497, 1.30951, 125833, 0, 0], '2019.02.26 00:00': [1.31167, 1.32879, 1.31141, 1.32491, 160731, 0, 0], '2019.02.27 00:00': [1.32513, 1.335, 1.32331, 1.33078, 152570, 0, 0], '2019.02.28 00:00': [1.33069, 1.33204, 1.32534, 1.32594, 135482, 0, 0], '2019.03.01 00:00': [1.32614, 1.32862, 1.31718, 1.32107, 136280, 0, 0], '2019.03.04 00:00': [1.32397, 1.32542, 1.31663, 1.31778, 141927, 0, 0], '2019.03.05 00:00': [1.31733, 1.31984, 1.30974, 1.31764, 144476, 0, 0], '2019.03.06 00:00': [1.31737, 1.31813, 1.31239, 1.31689, 132293, 0, 0], '2019.03.07 00:00

In [ ]:
response = Series(connector._get_response_())
responseData = Series(response["_data"])
print(type(responseData[0]))
order = connector._generate_default_order_dict()
print(order)

In [ ]:
order = connector._generate_default_order_dict()
connector._DWX_MTX_SEND_MARKETDATA_REQUEST_(_symbol='GBPUSD', _start=(Timestamp.now()-Timedelta(minutes=1)).strftime('%Y.%m.%d %H:%M:00'),_timeframe=1,_end=Timestamp.now().strftime('%Y.%m.%d %H:%M:00'))
response = Series(connector._get_response_())
responseData = Series(response["_data"])
order['_symbol'] = 'GBPUSD'
order['_price'] = responseData[0]
#connector._DWX_MTX_NEW_TRADE_(order)

In [ ]:
message = connector._DWX_MTX_GET_ALL_OPEN_TRADES_()
print(message)

In [ ]:
def obtainMarketDataChunk(symbol,startTime, endTime, timeframe=1, data='DATA', chunkSize = 125):
    outputFrame = pd.DataFrame()
    chunkTime = timeframe * chunkSize 
    
    pdStartTime = Timestamp.strptime(startTime,'%Y.%m.%d %H:%M:00')
    pdEndTime = Timestamp.strptime(endTime,'%Y.%m.%d %H:%M:00')
    timeDelta = pdEndTime - pdStartTime 
    divMod = divmod(timeDelta,Timedelta(minutes=chunkTime))
    currentStartTime = None
    currentEndTime = None
    response = None
    print(divMod)
    
    for x in range(divMod[0]):
        print("--------------------------------------------------------------------------------------------------")
        oldResponse = connector._get_response_()
        currentStartTime = pdStartTime + (x*Timedelta(minutes=(chunkTime)))
        currentEndTime = pdStartTime + (x+1)*Timedelta(minutes=(chunkTime))
        ID = connector._DWX_MTX_SEND_MARKETDATA_REQUEST_(_symbol=symbol, _start=currentStartTime.strftime('%Y.%m.%d %H:%M:00'),
                                                    _timeframe=timeframe,_end=currentEndTime.strftime('%Y.%m.%d %H:%M:00'))
        
        response = connector._get_response_(messageID = ID)
        outputFrame = outputFrame.append(Series(response['_data']).to_frame()) # Append to dataframe
        sleep(0.1) # To prevent resource timeout
        print("--------------------------------------------------------------------------------------------------")
        
    
    print("--------------------------------------------------------------------------------------------------")
    ID = connector._DWX_MTX_SEND_MARKETDATA_REQUEST_(_symbol=symbol, _start=currentEndTime.strftime('%Y.%m.%d %H:%M:00'),
                                                _timeframe=timeframe,_end=endTime)

    response = connector._get_response_(messageID = ID)
    outputFrame = outputFrame.append(Series(response['_data']).to_frame())
    outputFrame = outputFrame.reset_index()
    outputFrame.rename(columns={'index':'Date', 0:'ClosePrice'}, inplace=True)
    outputFrame['Date'] = pd.to_datetime(outputFrame['Date'], format='%Y.%m.%d %H:%M')
    outputFrame = outputFrame.set_index('Date')
    print("--------------------------------------------------------------------------------------------------")
    return outputFrame


In [ ]:
datachunk = obtainMarketDataChunk('GBPUSD', '2017.05.23 10:00:00', '2019.05.23 01:00:00',timeframe=1440)
print(datachunk.to_string())

In [ ]:
datachunk.plot()